## imports

In [2]:
import os
import sys
from pathlib import Path
from pprint import pformat
from tempfile import TemporaryDirectory
from datetime import datetime, timedelta
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import numpy as np
import geopandas as gpd
import pooch
import flopy
import flopy.plot
import flopy.utils
import rasterio
from rasterio.features import rasterize
from shapely.geometry import box
import contextily as cx
import re
from shapely.geometry import Point
from multiprocessing import Pool, cpu_count
import time
print(sys.version)
print(f"numpy version: {np.__version__}")
print(f"matplotlib version: {mpl.__version__}")
print(f"flopy version: {flopy.__version__}")

3.12.8 (main, Dec  4 2024, 14:07:02) [GCC 14.2.0]
numpy version: 1.26.4
matplotlib version: 3.10.0
flopy version: 3.9.2


## import model sim, heads, and budget

In [4]:
# Set up simulation parameters
sim_name = 'rgtihm'
workspace = './model'

# Start timing
start_time = time.time()

# Load the simulation
sim = flopy.mf6.MFSimulation.load(sim_name=sim_name, sim_ws=workspace)
gwf = sim.get_model(sim_name)

# Define file paths
head_file = os.path.join(workspace, f"{sim_name}.hds")
cbc_file = os.path.join(workspace, f"{sim_name}.cbc")

# Load output data
hds = flopy.utils.HeadFile(head_file)
head_data = hds.get_data()  # Last time step by default

cbc = flopy.utils.CellBudgetFile(cbc_file)
spdis = cbc.get_data(text='DATA-SPDIS')[-1]  # Specific discharge, last time step

# Get budget data (alternative approach for older FloPy versions)
budget_records = cbc.list_records()  # List all budget terms
budget_names = [rec[1].decode('utf-8') for rec in budget_records]
budget_values = []
for name in budget_names:
    data = cbc.get_data(text=name)
    if data:  # Check if data exists for this term
        budget_values.append(float(data[-1].sum()))  # Sum the last time step
    else:
        budget_values.append(0.0)

# Calculate and print runtime
runtime = time.time() - start_time
print(f"Simulation and output files loaded successfully. Runtime: {runtime:.2f} seconds")

loading simulation...
  loading simulation name file...
  loading tdis package...
  loading model gwf6...
    loading package dis...
    loading package ic...
    loading package oc...
    loading package ghb...
    loading package npf...
    loading package sto...
    loading package hfb...
    loading package uzf...
  loading solution package rgtihm...


/home/abdazzam/usr/local/lib/python3.12/site-packages/flopy/utils/binaryfile/__init__.py:1433: DeprecationWarning: list_records() is deprecated; use headers instead.
  warnings.warn(


(1, 1, b'          STO-SS', 328, 912, -9, 1, 15.5, 15.5, 15.5, b'', b'', b'', b'')
(1, 1, b'    FLOW-JA-FACE', 5380674, 1, -1, 1, 15.5, 15.5, 15.5, b'', b'', b'', b'')
(1, 1, b'      DATA-SPDIS', 328, 912, -9, 6, 15.5, 15.5, 15.5, b'RGTIHM          ', b'NPF             ', b'RGTIHM          ', b'NPF             ')
(1, 1, b'             GHB', 328, 912, -9, 6, 15.5, 15.5, 15.5, b'RGTIHM          ', b'RGTIHM          ', b'RGTIHM          ', b'GHB             ')
(2, 1, b'          STO-SS', 328, 912, -9, 1, 15.5, 31., 31., b'', b'', b'', b'')
(2, 1, b'    FLOW-JA-FACE', 5380674, 1, -1, 1, 15.5, 31., 31., b'', b'', b'', b'')
(2, 1, b'      DATA-SPDIS', 328, 912, -9, 6, 15.5, 31., 31., b'RGTIHM          ', b'NPF             ', b'RGTIHM          ', b'NPF             ')
(2, 1, b'             GHB', 328, 912, -9, 6, 15.5, 31., 31., b'RGTIHM          ', b'RGTIHM          ', b'RGTIHM          ', b'GHB             ')
(1, 2, b'          STO-SS', 328, 912, -9, 1, 15., 15., 46., b'', b'', b'', b'')
(1, 

TypeError: 'NoneType' object is not iterable

In [ ]:
# Extract grid coordinates from DIS package
xll = dis.xorigin.get_data()  # Lower-left x (feet)
yll = dis.yorigin.get_data()  # Lower-left y (feet)
angrot = dis.angrot.get_data()  # Rotation angle in degrees
delr = dis.delr.get_data()  # Cell widths (array in feet)
delc = dis.delc.get_data()  # Cell heights (array in feet)
ncol = dis.ncol.get_data()  # Number of columns
nrow = dis.nrow.get_data()  # Number of rows
idomain = dis.idomain.get_data()  # 3D IDOMAIN array

# Compute rotated grid coordinates (edges, not centers, for extent)
x = np.arange(ncol + 1) * delr[0]  # +1 for edges (in feet)
y = np.arange(nrow + 1) * delc[0]  # +1 for edges (in feet)
X, Y = np.meshgrid(x, y)
X_rot = xll + X * np.cos(np.radians(angrot)) - Y * np.sin(np.radians(angrot))  # in feet
Y_rot = yll + X * np.sin(np.radians(angrot)) + Y * np.cos(np.radians(angrot))  # in feet

# Convert coordinates to meters for plotting (EPSG:26913 UTM Zone 13N is in meters)
X_rot_m = X_rot * 0.3048  # Convert feet to meters
Y_rot_m = Y_rot * 0.3048  # Convert feet to meters
xll_m = xll * 0.3048  # Convert xll to meters
yll_m = yll * 0.3048  # Convert yll to meters

# Load active area shapefile
gdf = gpd.read_file('./shps/active_area.shp')
if gdf.crs != model_crs:
    gdf = gdf.to_crs(model_crs)

# 1. Overall model domain with IDOMAIN and shapefile
fig, ax = plt.subplots(figsize=(10, 10))
# Use IDOMAIN for layer 0 (Python index 0, MODFLOW layer 1)
ax.imshow(idomain[0], extent=[X_rot_m.min(), X_rot_m.max(), Y_rot_m.min(), Y_rot_m.max()], 
          cmap='binary', alpha=0.5)
gdf.plot(ax=ax, edgecolor='red', facecolor='none', linewidth=2)
cx.add_basemap(ax, crs=model_crs, source=cx.providers.OpenStreetMap.Mapnik)
ax.set_title('Model Domain with IDOMAIN (Layer 1) and active_area.shp')
ax.set_xlabel('Easting (m)')
ax.set_ylabel('Northing (m)')
plt.tight_layout()
plt.show()

# 2. Grid outline with IDOMAIN
fig, ax = plt.subplots(figsize=(10, 10))
cmap = plt.cm.colors.ListedColormap(['white', 'lightgrey'])
bounds = [0, 1, 2]
norm = plt.cm.colors.BoundaryNorm(bounds, cmap.N)
ax.imshow(idomain[0], extent=[X_rot_m.min(), X_rot_m.max(), Y_rot_m.min(), Y_rot_m.max()], 
          cmap=cmap, norm=norm, alpha=0.8, interpolation='nearest')
gdf.plot(ax=ax, edgecolor='lightgrey', facecolor='none', linewidth=2, alpha=0.5)

# Plot grid lines using edge coordinates in meters
for i in range(nrow + 1):  # +1 for edges
    ax.plot(X_rot_m[i, :], Y_rot_m[i, :], 'k-', lw=0.5)
for j in range(ncol + 1):  # +1 for edges
    ax.plot(X_rot_m[:, j], Y_rot_m[:, j], 'k-', lw=0.5)

cx.add_basemap(ax, crs=model_crs, source=cx.providers.OpenStreetMap.Mapnik)
ax.set_title('Grid with IDOMAIN (Layer 1)')
ax.set_xlabel('Easting (m)')
ax.set_ylabel('Northing (m)')
plt.tight_layout()
plt.show()